In [1]:
%matplotlib inline
%load_ext rpy2.ipython
import xarray as xr
import pandas as pd
import numpy as np
import os
import glob
import netCDF4
import matplotlib.pyplot as plt
import warnings
from rpy2.robjects import numpy2ri
from rpy2.robjects.packages import STAP
from rpy2.rinterface_lib.embedded import RRuntimeError

warnings.simplefilter("ignore")
plt.style.use("ggplot")
numpy2ri.activate()

# Tercile plots for GOTM output

This notebook compares results from GOTM using the **ERA5** data as "pseudo-observations" against predictions made using **S5** forecasts for each season in each year (with a 13 month warm-up period). It is based on the notebook *gotm_era_interim_s4_terciles.ipynb* that did essentailly the same using ERA-Interim and S4.

The aim is to create tercile plots for each season in each year for three variables: temperature, ice thickness and surface mixed layer depth.

## 1. User input

In [2]:
# Define seasons
seasons_dict = {
    "winter": [12, 1, 2],
    "spring": [3, 4, 5],
    "summer": [6, 7, 8],
    "autumn": [9, 10, 11],
}

# Years of interest
st_yr = 1994
end_yr = 2016

# Number of members in modelled data
members = 25

# Paths to datasets
store_obs_nc_path = (
    r"/home/jovyan/shared/WATExR/ERA5/Vansjo_simplyQ_gotm/store_full_era5.nc"
)
vanem_obs_nc_path = (
    r"/home/jovyan/shared/WATExR/ERA5/Vansjo_simplyQ_gotm/vanem_full_era5.nc"
)

store_mod_fold = r"/home/jovyan/shared/WATExR/ERA5/Vansjo_simplyQ_gotm/store"
vanem_mod_fold = r"/home/jovyan/shared/WATExR/ERA5/Vansjo_simplyQ_gotm/vanem"

# Folder in which to save tercile PDFs
terc_fold = r"./era5_s5_terc_plots/"

# Path dict with values as (obs, mod)
path_dict = {
    "store": (store_obs_nc_path, store_mod_fold),
    "vanem": (vanem_obs_nc_path, vanem_mod_fold),
}

## 2. Utility functions

In [3]:
def gotm_vars_to_dataframe(ds, basin):
    """ Processes/aggregates the different variables of interest and returns a dataframe
        with one column per variable.
        
    Args:
        ds:    Xarray dataset of GOTM output
        basin: Str. 'store' or 'vanem'
        
    Returns:
        Dataframe
    """
    from netCDF4 import Dataset

    # xarray can't read real depths from GOTM output
    # https://github.com/pydata/xarray/issues/2368#issuecomment-443218629
    # patch using netcdf4
    ds_path = (
        f"/home/jovyan/shared/WATExR/ERA5/Vansjo_simplyQ_gotm/{basin}_full_era5.nc"
    )
    ds2 = Dataset(ds_path, "r", format="NETCDF4")
    ds["z"] = ds2["z"][0, :, 0, 0]

    if basin == "vanem":
        # Extracts temperatures for depths of interest
        df = (
            ds["temp"].sel(z=[0, -6, -15], method="nearest").to_dataframe().unstack("z")
        )
        df.index = df.index.get_level_values("time")
        df.columns = ["0m", "6m", "15m"]

    elif basin == "store":
        # Get streamflow
        # 'int_water_balance' is water volume accumulated through the entire run.
        # To get flow, take first difference w.r.t. 'time' axis, then divide by 24*60*60
        ds["streamflow"] = ds["int_water_balance"].diff("time") / (24 * 60 * 60)

        # Use means (overall depths where relevant) for now
        df = ds[["streamflow"]].mean(dim=["lat", "lon"]).to_dataframe()

    else:
        raise ValueError("Basin not recognised.")

    df.index = pd.to_datetime(df.index.date)

    return df

As far as possible, I'd like to use the `visualizeR` functions for generating the pots, in order to be consistent with other work. I'll do most of the data processing in Python, then pass variables over to R and re-structure for the crucial last steps in each loop.

In [4]:
r_fct = """    
tercile_plot <- function(var_name, long_name, st_dates, end_dates, obs_values, mod_values, out_path, mod_template){

    # Function to reformat GOTM data for use in visualizeR's tercilePlot and FairRPSS functions.
    # Uses an example of the 'obs' and 'mod' output for a single variable saved
    # from the notebook here:
    #
    #     WATExR/Norway_Morsa/MetData_Processing/notebooks/03_validation_tercile_plots.ipynb
    #
    # Args:
    #     var_name:     Str. Short name for variable
    #     long_name:    Str. Long name for variable
    #     st_dates:     Numpy array of strings with dates in format 'YYYY-MM-DD'
    #     end_dates:    Numpy array of strings with dates in format 'YYYY-MM-DD'
    #     obs_values:   Numpy array of floats
    #     mod_values:   Numpy array of floats
    #     out_path:     Str. Path to PDF to be created
    #     mod_template: Str. Either 'mod_data_template_15_members.rda' or 'mod_data_template_25_members.rda',
    #                   depending on how many members you're using in 'mod_values'
    # 
    # Returns:
    #     Float vector c(FairRPSS, significance). Tercile plot PDF is saved to the path specified.

    suppressPackageStartupMessages(library(Cairo))
    suppressPackageStartupMessages(library(visualizeR))
    suppressPackageStartupMessages(library(easyVerification))
    
    # Convert dates to vectors with POSIX format
    st_dates <- c(st_dates)
    st_dates <- strptime(st_dates, "%Y-%m-%d")
    end_dates <- c(end_dates)
    end_dates <- strptime(end_dates, "%Y-%m-%d")

    # Obs values as R array
    obs_values <- array(obs_values)
    attr(obs_values, "dimensions") <- c("time")

    # Mod values as R array
    mod_values <- array(mod_values, dim = c(25, length(st_dates)))
    attr(mod_values, "dimensions") <- c("member", "time")

    # Load data "templates" from original tercile plot script
    obs_temp <- get(load("obs_data_template.rda"))
    mod_temp <- get(load(mod_template))

    # Update obs "template" with new data
    obs_temp$Variable$varName <- var_name
    attr(obs_temp$Variable, 'longname') <- long_name
    obs_temp$Data <- obs_values
    obs_temp$Dates$start <- st_dates
    obs_temp$Dates$end <- end_dates

    # Update mod "template" with new data
    mod_temp$Variable$varName <- var_name
    attr(mod_temp$Variable, 'longname') <- long_name
    mod_temp$Data <- mod_values
    mod_temp$Dates$start <- st_dates
    mod_temp$Dates$end <- end_dates
    mod_temp$InitializationDates <- NULL

    # Make tercile plot
    CairoPDF(file = out_path, width = 10)
    tercilePlot(obs = redim(obs_temp), hindcast = redim(mod_temp))
    dev.off()
    
    # Calculate FairRPSS
    frpss <- veriApply(verifun = "FairRpss",
                       fcst = mod_temp$Data,
                       obs = obs_temp$Data,
                       prob = c(1/3,2/3),
                       tdim = 2, 
                       ensdim = 1)
                       
    return (c(frpss$skillscore, frpss$skillscore.sd))
}
"""

# Load R function for making tercile plots
r_pkg = STAP(r_fct, "r_pkg")

## 3. Data processing

In [5]:
basin_list = []
season_list = []
variable_list = []
frpss_list = []
sig_list = []

# Loop over basins
for basin in path_dict.keys():
    print(f"Processing: {basin}fjorden")
    obs_path = path_dict[basin][0]
    mod_fold = path_dict[basin][1]

    # Read obs
    obs_ds = xr.open_dataset(obs_path, drop_variables="z")
    obs_df = gotm_vars_to_dataframe(obs_ds, basin)

    # Loop over seasons
    for season in seasons_dict.keys():
        print(f"  {season}")
        print(f"    Getting S5 data")

        mem_list = []

        # Loop over S5 members
        for mem in range(1, 26):
            # Get S5 data for member (for all years in this season)
            search_path = os.path.join(mod_fold, f"{basin}_*_{season}_{mem}.nc")
            flist = sorted(glob.glob(search_path))

            # Loop over files for each year
            year_list = []
            for fpath in flist:
                mem_yr_ds = xr.open_dataset(fpath, drop_variables="z")
                mem_yr_df = gotm_vars_to_dataframe(mem_yr_ds, basin)

                # Get just the months of interest
                months = seasons_dict[season]
                mem_yr_df = mem_yr_df.iloc[
                    -100:
                ]  # Ensures we don't gets months from the previous year "warm-up" too
                mem_yr_df["month"] = mem_yr_df.index.month
                mem_yr_df = mem_yr_df.query("month in @months")
                del mem_yr_df["month"]

                year_list.append(mem_yr_df)

            # Build single df for this member for all years
            mem_df = pd.concat(year_list, axis="rows")

            # Create multi-index with member
            iterables = [mem_df.columns, [mem + 1,]]
            mem_df.columns = pd.MultiIndex.from_product(
                iterables, names=["variable", "member"]
            )
            mem_list.append(mem_df)

        # Build a dataframe of all members
        mod_df = pd.concat(mem_list, axis="columns")

        # Subset to 1994 - 2016
        mod_df = mod_df[(mod_df.index.year >= st_yr) & (mod_df.index.year <= end_yr)]

        # Get matching dates from obs df
        obs_df_seas = obs_df[obs_df.index.isin(mod_df.index)]

        assert len(obs_df_seas) == len(
            mod_df
        ), f"Dataframe lengths do not match for {season}."

        # Loop over variables
        if basin == "store":
            variables = ["streamflow"]
        else:
            variables = ["0m", "6m", "15m"]

        for variable in variables:
            # Get data series
            obs = obs_df_seas[variable]
            mod = mod_df[variable]

            try:
                # Extract components for passing to R
                if variable == "streamflow":
                    long_name = "Daily average streamflow"
                else:
                    long_name = obs_ds["temp"].attrs["long_name"] + f" ({variable})"
                st_dates = obs.index.strftime("%Y-%m-%d").values
                end_dates = (
                    (obs.index + pd.DateOffset(days=1)).strftime("%Y-%m-%d").values
                )  # Shifted by 1 day from 'start'
                obs_values = obs.values
                mod_values = mod.values.T
                out_pdf = os.path.join(
                    terc_fold, f"gotm_era5_s5_terc_plot_{basin}_{variable}_{season}.pdf"
                )
                mod_template = f"mod_data_template_{members}_members.rda"

                # Make plot using R function
                frpss, sig = r_pkg.tercile_plot(
                    variable,
                    long_name,
                    st_dates,
                    end_dates,
                    obs_values,
                    mod_values,
                    out_pdf,
                    mod_template,
                )

                # Store FRPSS
                basin_list.append(basin)
                season_list.append(season)
                variable_list.append(variable)
                frpss_list.append(frpss)
                sig_list.append(sig)

                print(f"      Finished plot for: {variable}")

            except RRuntimeError:
                print(f"      Skipping {variable} as insufficient data.")

# Save FRPSS results
frpss_df = pd.DataFrame(
    {
        "basin": basin_list,
        "season": season_list,
        "varaible": variable_list,
        "frpss": frpss_list,
        "sig": sig_list,
    }
)
csv_path = r"./era5_s5_terc_plots/morsa_gotm_frpss.csv"
frpss_df.to_csv(csv_path, index=False)

Processing: storefjorden
  winter
    Getting S5 data

    _______   ____  ___________________  __  ________ 
   / ___/ /  / /  |/  / __  /_  __/ __/ / / / / __  / 
  / /  / /  / / /|_/ / /_/ / / / / __/ / /_/ / /_/_/  
 / /__/ /__/ / /  / / __  / / / / /__ /___  / / \ \ 
 \___/____/_/_/  /_/_/ /_/ /_/  \___/    /_/\/   \_\ 
 
      github.com/SantanderMetGroup/climate4R



R[write to console]: Please note that FairRpss is computed without ensemble-size correction for the climatological reference forecast, as categories are based on quantiles of the observations and forecast probabilities are therefore known.



  |                                                  | 0 % ~calculating   |++++++++++++++++++++++++++++++++++++++++++++++++++| 100% elapsed=16s  
      Finished plot for: streamflow
  spring
    Getting S5 data


R[write to console]: Please note that FairRpss is computed without ensemble-size correction for the climatological reference forecast, as categories are based on quantiles of the observations and forecast probabilities are therefore known.



  |                                                  | 0 % ~calculating   |++++++++++++++++++++++++++++++++++++++++++++++++++| 100% elapsed=16s  
      Finished plot for: streamflow
  summer
    Getting S5 data


...

  |                                                  | 0 % ~calculating   |++++++++++++++++++++++++++++++++++++++++++++++++++| 100% elapsed=13s  
      Finished plot for: 0m


R[write to console]: Please note that FairRpss is computed without ensemble-size correction for the climatological reference forecast, as categories are based on quantiles of the observations and forecast probabilities are therefore known.



  |                                                  | 0 % ~calculating   |++++++++++++++++++++++++++++++++++++++++++++++++++| 100% elapsed=15s  
      Finished plot for: 6m


R[write to console]: Please note that FairRpss is computed without ensemble-size correction for the climatological reference forecast, as categories are based on quantiles of the observations and forecast probabilities are therefore known.



  |                                                  | 0 % ~calculating   |++++++++++++++++++++++++++++++++++++++++++++++++++| 100% elapsed=14s  
      Finished plot for: 15m


## 4. Checking

The code below generates a tercile plot using Python, based on my understanding of what the `visualizeR` function is doing. Note that the code below **doesn't make allowances for winter months crossing the year-end boundary**, so it is not expected to produce correct results for winter. However, comparisons for the other seasons seem OK and are enough to convince me that my Python <-> R interaction are working reasonabley.

In [ ]:
print(basin, variable, season)

In [ ]:
obs_ann = obs.groupby(obs.index.year).mean()
obs_quants = obs_ann.quantile([0.33, 0.67])
obs_quants

In [ ]:
# Build bins for terciles
terc_bins = obs_quants.values
terc_bins = np.insert(terc_bins, 0, -np.inf)
terc_bins = np.append(terc_bins, np.inf)
terc_bins

In [ ]:
mod_ann = mod.groupby(mod.index.year).mean()
mod_ann.head()

In [ ]:
# Get terciles
df_list = []
for year in mod_ann.index:
    terc_df = pd.cut(
        mod_ann.loc[year],
        bins=terc_bins,
        labels=["Below normal", "Near normal", "Above normal"],
    )

    terc_prob = 100 * terc_df.value_counts() / 25
    df_list.append(terc_prob)

df = pd.concat(df_list, axis="columns").T

df.head()

In [ ]:
data = df.T.iloc[::-1].values
fig = plt.figure(figsize=(20, 6))
plt.imshow(
    data, cmap="gray_r", extent=[st_yr - 0.5, end_yr + 0.5, 0.5, 3.5], vmin=0, vmax=100
)
plt.colorbar(orientation="horizontal", fraction=0.05, aspect=50)

##  Appendix

The following "interactive" code was used to develop and test the R `tercile_plot` function.

In [ ]:
# # Get data for testing
# obs = obs_df_seas["0m"]
# mod = mod_df["0m"]

# # Extract components for passing to R
# var_name = '0m'
# long_name = 'Temperature'
# st_dates = obs.index.strftime('%Y-%m-%d').values
# end_dates = (obs.index + pd.DateOffset(days=1)).strftime('%Y-%m-%d').values
# obs_values = obs.values
# mod_values = mod.values.T

In [ ]:
# %%R -i var_name,long_name,st_dates,end_dates,obs_values,mod_values -o mod_temp,obs_temp

# suppressPackageStartupMessages(library(Cairo))
# suppressPackageStartupMessages(library(visualizeR))

# st_dates <- c(st_dates)
# st_dates <- strptime(st_dates, "%Y-%m-%d")
# end_dates <- c(end_dates)
# end_dates <- strptime(end_dates, "%Y-%m-%d")

# obs_values <- array(obs_values)
# attr(obs_values, "dimensions") <- c("time")

# mod_values <- array(mod_values, dim = c(25, length(st_dates)))
# attr(mod_values, "dimensions") <- c("member", "time")

# obs_temp <- get(load("obs_data_template.rda"))
# mod_temp <- get(load("mod_data_template_25_members.rda"))

# obs_temp$Variable$varName <- var_name
# attr(obs_temp$Variable, 'longname') <- long_name
# obs_temp$Data <- obs_values
# obs_temp$Dates$start <- st_dates
# obs_temp$Dates$end <- end_dates

# mod_temp$Variable$varName <- var_name
# attr(mod_temp$Variable, 'longname') <- long_name
# mod_temp$Data <- mod_values
# mod_temp$Dates$start <- st_dates
# mod_temp$Dates$end <- end_dates
# mod_temp$InitializationDates <- NULL

# CairoPDF(file = 'test', width = 10)
# tercilePlot(obs = redim(obs_temp), hindcast = redim(mod_temp))